In [129]:
# ## Imports

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import time

tfd = tfp.distributions

In [ ]:
# ## Start time measures

start_time = time.clock()

In [130]:
# ## Reproducibility
# 
# Seed setting for reproducable research.

#Set numpy seed
np.random.seed(1234)

# Set graph-level seed
tf.set_random_seed(1234)

In [131]:
# ## Distributions functions
# 


def runif(lower, higher):
    dist = tfd.Uniform(lower, higher)
    return dist.sample()

def rnorm(mean, var):
    sd = tf.sqrt(var)
    dist = tfd.Normal(loc= mean, scale= sd)
    return dist.sample()

def rbeta(alpha, beta):
    dist = tfd.Beta(alpha, beta)
    return dist.sample()

def rinvchisq(df, scale):
    dist = tfd.InverseGamma(df*0.5, df*scale*0.5)
    return dist.sample()

def rbernoulli(p):
    dist = tfd.Bernoulli(probs=p)
    return dist.sample()


# ## Sampling functions
# 

# sample mean
def sample_mu(N, Esigma2, Y, X, beta): #as in BayesC, with the N parameter
    mean = tf.reduce_sum(tf.subtract(Y, tf.matmul(X, beta)))/N
    sd = tf.sqrt(Esigma2/N)
    mu = rnorm(mean, sd)
    return mu

# sample variance of beta
def sample_psi2_chisq( beta, NZ, v0B, s0B):
    df=v0B+NZ
    scale=(tf.nn.l2_loss(beta)*2*NZ+v0B*s0B)/(v0B+NZ)
    psi2=rinvchisq(df, scale)
    return psi2


# sample error variance of Y
def sample_sigma_chisq( N, epsilon, v0E, s0E):
    sigma2=rinvchisq(v0E+N, (tf.nn.l2_loss(epsilon)*2+v0E*s0E)/(v0E+N))
    return(sigma2)


# sample mixture weight
def sample_w( M, NZ):
    w=rbeta(1+NZ,1+(M-NZ))
    return w

In [143]:
## Simulate data

# Var(g) = 0.7
# Var(Beta_true) = Var(g) / M
# Var(error) = 1 - Var(g) 


def build_toy_dataset(N, M, var_g):
    
    sigma_b = np.sqrt(var_g/M)
    sigma_e = np.sqrt(1 - var_g)
    
    beta_true = np.random.normal(0, sigma_b , M)
    x = sigma_b * np.random.randn(N, M) 
    y = np.dot(x, beta_true) + np.random.normal(0, sigma_e, N)
    return x, y, beta_true

# Simulated data parameters

N = 100       # number of data points
M = 10        # number of features
var_g = 0.7   # M * var(Beta_true)


x, y, beta_true = build_toy_dataset(N, M, var_g)

X = tf.constant(x, shape=[N,M], dtype=tf.float32)
Y = tf.constant(y, shape = [N,1], dtype=tf.float32)

In [144]:
# ## Parameters setup

# Distinction between constant and variables
# Variables: values might change between evaluation of the graph
# (if something changes within the graph, it should be a variable)

Emu = tf.Variable(0., dtype=tf.float32 , trainable=False)
vEmu = tf.ones([N,1])
Ebeta = np.zeros([M,1])
Ebeta_ = tf.Variable(Ebeta, dtype=tf.float32, trainable=False)
ny = np.zeros([M,1])
Ew = tf.Variable(0., trainable=False)
epsilon = tf.Variable(Y, trainable=False)
NZ = tf.Variable(0., trainable=False)
Esigma2 = tf.Variable(tf.nn.l2_loss(epsilon.initialized_value())/N, trainable=False)
Epsi2 = tf.Variable(rbeta(1.,1.), trainable=False)
Cj = tf.Variable(0.0, dtype=tf.float32, trainable=False)
rj = tf.Variable(0.0, dtype=tf.float32, trainable=False)
ratio = tf.Variable(0.0, dtype=tf.float32, trainable=False)
pij = tf.Variable(0.0, dtype=tf.float32, trainable=False)





#Standard parameterization of hyperpriors for variances
v0E=tf.constant(0.001)
s0E=tf.constant(0.001)
v0B=tf.constant(0.001)
s0B=tf.constant(0.001)



# ## Tensorboard graph

#writer = tf.summary.FileWriter('.')
#writer.add_graph(tf.get_default_graph())


In [145]:
# Open session
sess = tf.Session()


# In[86]:


# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

In [141]:
test = sess

RuntimeError: Attempted to use a closed Session.

In [140]:
sess.close()

In [142]:
tf.reset_default_graph()


In [ ]:
num_iter = 500


# In[88]:


# update ops
# u_epsilon_add = epsilon.assign(tf.add(epsilon, tf.reshape(X[:,marker]*Ebeta[marker],[N,1])))
# u_epsilon_sub = epsilon.assign(tf.subtract(epsilon, tf.reshape(X[:,marker]*Ebeta[0],[N,1])))
# u_Ebeta_ = Ebeta_.assign(Ebeta)
# u_epsilon = epsilon.assign(Y-tf.matmul(X,Ebeta_)-vEmu*Emu)
# u_Emu = Emu.assign(sample_mu(N, Esigma2, Y, X, Ebeta_))
# u_NZ = NZ.assign(np.sum(ny))
# u_Ew = Ew.assign(sample_w(M,NZ))
# u_epsi2 = Epsi2.assign(sample_psi2_chisq(Ebeta_,NZ,v0B,s0B))
# u_Esigma2 = Esigma2.assign(sample_sigma_chisq(N,epsilon,v0E,s0E))


# In[ ]:


for i in range(num_iter):
    index = np.random.permutation(M)
    print("Gibbs sampling iteration: ", i)
    sess.run(Emu.assign(sample_mu(N, Esigma2, Y, X, Ebeta_)))
    for marker in index:
        print(marker)
        sess.run(epsilon.assign_add(tf.reshape(X[:,marker]*Ebeta[marker],[N,1])))
        sess.run(Cj.assign(tf.reduce_sum(tf.pow(X[:,0],2)) + Esigma2/Epsi2)) #adjusted variance
        sess.run(rj.assign(tf.matmul(tf.reshape(X[:,marker], [1,N]),epsilon)[0][0])) # mean
        sess.run(ratio.assign(tf.exp(-(tf.pow(rj,2))/(2*Cj*Esigma2))*tf.sqrt((Epsi2*Cj)/Esigma2)))
        sess.run(pij.assign(Ew/(Ew+ratio*(1-Ew))))

        ny[marker] = sess.run(rbernoulli(pij))

        if (ny[marker]==0):
            Ebeta[marker]=0

        elif (ny[marker]==1):
            Ebeta[marker]=sess.run(rnorm(rj/Cj,Esigma2/Cj))

        sess.run(epsilon.assign_sub(tf.reshape(X[:,marker]*Ebeta[marker],[N,1])))

    #for i in range(len(Ebeta)):
    #    print(Ebeta[i], "\t", ny[i])
    sess.run(NZ.assign(np.sum(ny)))
    sess.run(Ew.assign(sample_w(M,NZ)))
    sess.run(Ebeta_.assign(Ebeta))
    sess.run(epsilon.assign(Y-tf.matmul(X,Ebeta_)-vEmu*Emu))
    sess.run(Epsi2.assign(sample_psi2_chisq(Ebeta_,NZ,v0B,s0B)))
    sess.run(Esigma2.assign(sample_sigma_chisq(N,epsilon,v0E,s0E)))
    
    
    
#     sess.run(u_Ebeta_)
#     sess.run(u_NZ)
#     sess.run(u_Ew)
#     sess.run(u_epsilon)
#     sess.run(u_epsi2)
#     sess.run(u_Esigma2)

# ## End session
sess.close()

# ## Print results
print("Ebeta" + "\t" + ' ny' + '\t'+ ' beta_true')
for i in range(M):
    print(Ebeta[i], "\t", ny[i], "\t", beta_true[i])


# ## Printe time
print('Time elapsed: ')
print(time.clock() - start_time, "seconds")